In [4]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pybaseball import schedule_and_record
from pybaseball import cache
from pybaseball import batting_stats

cache.enable()

In [5]:
data = batting_stats(2019)

In [6]:
data

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,L-WAR
1,10155,2019,Mike Trout,LAA,27,134,470,600,137,63,...,116.6,155,0.438,354,0.193,0.258,0.310,0.678,0.460,8.4
2,17678,2019,Alex Bregman,HOU,25,156,554,690,164,84,...,107.4,178,0.372,479,0.204,0.250,0.263,0.453,0.373,8.4
4,15998,2019,Cody Bellinger,LAD,23,156,558,660,170,86,...,110.6,207,0.456,454,0.149,0.246,0.319,0.635,0.430,7.7
0,11477,2019,Christian Yelich,MIL,27,130,489,580,161,85,...,117.9,182,0.487,374,0.140,0.258,0.314,0.629,0.429,7.8
5,12861,2019,Anthony Rendon,WSN,29,146,545,646,174,93,...,107.7,218,0.466,468,0.172,0.223,0.310,0.590,0.414,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,1744,2019,Miguel Cabrera,DET,36,136,493,549,139,106,...,111.4,173,0.444,390,0.151,0.260,0.270,0.436,0.331,-0.3
115,1177,2019,Albert Pujols,LAA,39,131,491,545,120,75,...,112.3,155,0.360,431,0.197,0.272,0.251,0.428,0.319,-0.6
114,3516,2019,Eric Hosmer,SDP,29,160,619,667,164,111,...,114.3,212,0.460,461,0.145,0.280,0.249,0.411,0.304,-0.3
130,9112,2019,Khris Davis,OAK,31,133,481,533,106,72,...,109.9,133,0.395,337,0.103,0.272,0.232,0.437,0.318,-0.9
